In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from pyriemann.clustering import Kmeans

In [ ]:
data = pd.read_csv('tb2_data_clean.csv', sep=';')
data.head()

In [ ]:
data = data.iloc[:,1:3]
data

In [ ]:
data.values

In [ ]:
Sum_of_squared_distances = []
K = range(1,10)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(data.values)
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
k_means = KMeans(n_clusters=3)
k_means.fit(data.values)
clusters = k_means.fit_predict(data.values)

In [ ]:
plt.scatter(data.values[:, 1], data.values[:, 0], c=clusters, s=50, cmap='viridis')

centers = k_means.cluster_centers_
plt.scatter(centers[:, 1], centers[:, 0], c='red', s=200, alpha=0.5);
plt.gca().set_box_aspect(0.5)

In [ ]:
def k_mean_distance(data, cx, cy, i_centroid, cluster_labels):
    distances = [np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[cluster_labels == i_centroid]]
    return distances

distances = []
for i, (cx, cy) in enumerate(centers):
    print(i, cx, cy)
    mean_distance = k_mean_distance(data.values, cx, cy, i, clusters)
    distances.append(mean_distance)
    print(min(distances[i]))
    print(max(distances[i]))
    print('-----------------------------------')

In [ ]:
jarak = pd.DataFrame(distances[0])


# scaling data
mms = MinMaxScaler()
mms.fit(jarak)
print(mms.data_min_)
print(mms.data_max_)
jarak_scaled = mms.transform(jarak)
print(jarak_scaled.min())
print(jarak_scaled.max())

In [ ]:
binwidth = 50

ax = plt.subplot(111)
ax.hist(jarak_scaled, bins = binwidth,
             color = 'blue', edgecolor = 'black')
ax.set_title('Bin = %d' % binwidth, size = 30)
ax.set_xlabel('Jarak', size = 25)
ax.set_ylabel('Data', size= 25)

plt.tight_layout()
plt.show()

In [ ]:
### ERROR ###

k_means_riemann = Kmeans(n_clusters=3, n_jobs=-2)
k_means_riemann.fit(data.values)
clusters_riemann = k_means_riemann.fit_predict(data.values)

In [ ]:
### SKIP ###

plt.scatter(data.values[:, 1], data.values[:, 0], c=clusters_riemann, s=50, cmap='viridis')

centers_riemann = k_means_riemann.cluster_centers_
plt.scatter(centers_riemann[:, 1], centers_riemann[:, 0], c='red', s=200, alpha=0.5);
plt.gca().set_box_aspect(0.5)

In [ ]:
### DBSCAN ###

db = DBSCAN(eps=0.3, min_samples=100).fit(data.values)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(data.values, labels))